In [2]:
import numpy as np 
import cv2
import matplotlib.pyplot as plt

import tensorflow as tf 
from tensorflow import keras
from tensorflow.keras import layers

import imageio

import os


2023-10-24 15:15:51.521274: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-24 15:15:51.521300: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-24 15:15:51.521326: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
batch_size = 32
num_channels = 3 # actually there are three channels in this image
num_classes = 1
image_size = (240,320)
latent_dim = 128

In [2]:
# frame_data_dir = '/home/bestlab/Desktop/Squishy-Methane-Analysis/MethaneModel/frame_data_movingAvg/'
frame_data_dir = '/home/bestlab/Desktop/Squishy-Methane-Analysis/0 - GasNet/frame_data_movingAvg/'
frame_train_data_dir = os.path.join(frame_data_dir, 'train')
frame_test_data_dir = os.path.join(frame_data_dir, 'test')
frame_train_nonleak_data_dir = os.path.join(frame_train_data_dir,'Nonleaks')
frame_train_leak_data_dir = os.path.join(frame_train_data_dir,'Leak')


## Load in data in training set

Here, we cannot load in both non leak and leak to generate pics, because they are originally inbalanced.  
Let's try to import all the non-leak pics and down sample the leak pics to create a balanced set for GAN.

In [37]:
nonleak_frame_dataset = tf.keras.utils.image_dataset_from_directory(
    directory=frame_train_nonleak_data_dir,
    labels=None,
    batch_size=32,
    image_size=(240,320),
    shuffle=True,
    seed=42,
)
leak_frame_dataset = tf.keras.utils.image_dataset_from_directory(
    directory = frame_train_leak_data_dir,
    labels=None,
    batch_size=32,
    image_size=(240,320),
    shuffle=True,
    seed=42,
)

Found 116180 files belonging to 1 classes.
Found 163893 files belonging to 1 classes.


### Down sample the leak set to the size of nonleak

In [6]:
n_nonleak = len(nonleak_frame_dataset)
print(n_nonleak)
leak_frame_dataset_downsampled = leak_frame_dataset.take(n_nonleak)
print(len(leak_frame_dataset_downsampled))

3631
3631


In [12]:
# plt.figure(figsize=(10, 10))
# for images in leak_frame_dataset_downsampled.take(1):
#     for i in range(9):
#         ax = plt.subplot(3,3, i + 1)
#         plt.imshow(images[i].numpy().astype("uint8"))
#         plt.title('leak')
#         plt.tight_layout()
#         plt.axis("off")

In [13]:
# plt.figure(figsize=(10, 10))
# for images in nonleak_frame_dataset.take(1):
#     for i in range(9):
#         ax = plt.subplot(3,3, i + 1)
#         plt.imshow(images[i].numpy().astype("uint8"))
#         plt.title('nonleak')
#         plt.tight_layout()
#         plt.axis("off")

In [7]:
all_digits = nonleak_frame_dataset.concatenate(leak_frame_dataset_downsampled)
def generate_label(label,batch_size,dataset_size):
    temp = np.array([[label]*batch_size]*dataset_size)
    temp = temp.reshape((-1,32,1))
    return temp.astype('float32')

label_nonleak = tf.data.Dataset.from_tensor_slices(generate_label(0,32,n_nonleak))
label_leak = tf.data.Dataset.from_tensor_slices(generate_label(1,32,n_nonleak))
all_labels = label_nonleak.concatenate(label_leak)

dataset = tf.data.Dataset.zip((all_digits,all_labels))

## Calculating the number of input channel for the generator and discriminator

In [8]:
generator_in_channels = latent_dim + num_classes
discriminator_in_channels = num_channels + num_classes
print(generator_in_channels, discriminator_in_channels)

129 4


## Creating the discriminator and generator


In [9]:
# Create the discriminator.
discriminator = keras.Sequential(
    [
        keras.layers.InputLayer((240,320, discriminator_in_channels)),
        layers.Conv2D(64, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(128, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.GlobalMaxPooling2D(),
        layers.Dense(1),
    ],
    name="discriminator",
)

# Create the generator.
generator = keras.Sequential(
    [
        keras.layers.InputLayer((generator_in_channels,)),

        layers.Dense(15 * 20 * generator_in_channels),
        layers.LeakyReLU(alpha=0.2),
        layers.Reshape((15, 20, generator_in_channels)),
        
        layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        
        layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        
        layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        
        layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        
        layers.Conv2D(3, (7, 7), padding="same", activation="sigmoid"),
    ],
    name="generator",
)

## Creating a ConditionalGAN model


In [10]:
class ConditionalGAN(keras.Model):
    def __init__(self, discriminator, generator, latent_dim):
        super().__init__()
        self.discriminator = discriminator
        self.generator = generator
        self.latent_dim = latent_dim
        self.gen_loss_tracker = keras.metrics.Mean(name="generator_loss")
        self.disc_loss_tracker = keras.metrics.Mean(name="discriminator_loss")
        
    def get_config(self):
            return {
                "discriminator_config": self.discriminator.get_config(),
                "generator_config": self.generator.get_config(),
                "latent_dim": self.latent_dim,
            }

    @classmethod
    def from_config(cls, config,discriminator,generator):
        discriminator = discriminator.from_config(config["discriminator_config"])
        generator = generator.from_config(config["generator_config"])
        return cls(discriminator=discriminator, generator=generator, latent_dim=config["latent_dim"])


    @property
    def metrics(self):
        return [self.gen_loss_tracker, self.disc_loss_tracker]

    def compile(self, d_optimizer, g_optimizer, loss_fn):
        super().compile()
        self.d_optimizer = d_optimizer
        self.g_optimizer = g_optimizer
        self.loss_fn = loss_fn

    def call(self, data):
        x = self.forward(data)
        return x
    
    def forward(self,data):
        # Unpack the data.
        real_images, one_hot_labels = data
        # Add dummy dimensions to the labels so that they can be concatenated with
        # the images. This is for the discriminator.
        image_one_hot_labels = one_hot_labels[:, :, None, None]
        image_one_hot_labels = tf.repeat(
            image_one_hot_labels, repeats=[240 * 320]
        )
        image_one_hot_labels = tf.reshape(
            image_one_hot_labels, (-1, 240, 320, num_classes)
        )

        # Sample random points in the latent space and concatenate the labels.
        # This is for the generator.
        batch_size = tf.shape(real_images)[0]
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))
        random_vector_labels = tf.concat(
            [random_latent_vectors, one_hot_labels], axis=1
        )

        # Decode the noise (guided by labels) to fake images.
        generated_images = self.generator(random_vector_labels)

        # Combine them with real images. Note that we are concatenating the labels
        # with these images here.
        fake_image_and_labels = tf.concat([generated_images, image_one_hot_labels], -1)
        real_image_and_labels = tf.concat([real_images, image_one_hot_labels], -1)

        combined_images = tf.concat(
            [fake_image_and_labels, real_image_and_labels], axis=0
        )

        return combined_images,image_one_hot_labels
    
    def train_step(self, data):

        real_images, one_hot_labels = data
        batch_size = tf.shape(real_images)[0]

        combined_images, image_one_hot_labels = self.forward(data)
        
        # Assemble labels discriminating real from fake images.
        labels = tf.concat(
            [tf.ones((batch_size, 1)), tf.zeros((batch_size, 1))], axis=0
        )

        # Train the discriminator.
        with tf.GradientTape() as tape:
            predictions = self.discriminator(combined_images)
            d_loss = self.loss_fn(labels, predictions)
        grads = tape.gradient(d_loss, self.discriminator.trainable_weights)
        self.d_optimizer.apply_gradients(
            zip(grads, self.discriminator.trainable_weights)
        )

        # Sample random points in the latent space.
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))
        random_vector_labels = tf.concat(
            [random_latent_vectors, one_hot_labels], axis=1
        )

        # Assemble labels that say "all real images".
        misleading_labels = tf.zeros((batch_size, 1))

        # Train the generator (note that we should *not* update the weights
        # of the discriminator)!
        with tf.GradientTape() as tape:
            fake_images = self.generator(random_vector_labels)
            fake_image_and_labels = tf.concat([fake_images, image_one_hot_labels], -1)
            predictions = self.discriminator(fake_image_and_labels)
            g_loss = self.loss_fn(misleading_labels, predictions)
        grads = tape.gradient(g_loss, self.generator.trainable_weights)
        self.g_optimizer.apply_gradients(zip(grads, self.generator.trainable_weights))

        # Monitor loss.
        self.gen_loss_tracker.update_state(g_loss)
        self.disc_loss_tracker.update_state(d_loss)
        return {
            "g_loss": self.gen_loss_tracker.result(),
            "d_loss": self.disc_loss_tracker.result(),
        }

## Train a GAN Model

In [10]:
cond_gan = ConditionalGAN(
    discriminator=discriminator, generator=generator, latent_dim=latent_dim
)
cond_gan.compile(
    d_optimizer=keras.optimizers.Adam(learning_rate=0.0003),
    g_optimizer=keras.optimizers.Adam(learning_rate=0.0003),
    loss_fn=keras.losses.BinaryCrossentropy(from_logits=True),
)

for real_images, one_hot_labels in dataset.take(1):
    cond_gan((real_images,one_hot_labels))


In [ ]:
cond_gan.fit(dataset, epochs=20)

In [13]:
assert(1==3)
cond_gan.save('./models/oct12-gan/')

AssertionError: 

In [3]:
path = './models/oct12-gan/'
cond_gan = tf.keras.models.load_model(path)

## Generating fake nonleak images


In [17]:
# We first extract the trained generator from our Conditional GAN.
trained_gen = cond_gan.generator
number_nonleak = 42881
number_leak = 304911
number_diff = number_leak-number_nonleak
number_gen = number_nonleak
# Choose the number of intermediate images that would be generated in
# between the interpolation + 2 (start and last images).
num_images = number_gen  # @param {type:"integer"}

# Sample noise for the interpolation.
interpolation_noise = tf.random.normal(shape=(1, latent_dim))
interpolation_noise = tf.repeat(interpolation_noise, repeats=num_images)
interpolation_noise = tf.reshape(interpolation_noise, (num_images, latent_dim))

In [7]:


# def interpolate_class(first_number, second_number):
#     # Convert the start and end labels to one-hot encoded vectors.
#     first_label = keras.utils.to_categorical([first_number], num_classes)
#     second_label = keras.utils.to_categorical([second_number], num_classes)
#     first_label = tf.cast(first_label, tf.float32)
#     second_label = tf.cast(second_label, tf.float32)

#     # Calculate the interpolation vector between the two labels.
#     percent_second_label = tf.linspace(0, 1, num_images)[:, None]
#     percent_second_label = tf.cast(percent_second_label, tf.float32)
#     interpolation_labels = (
#         first_label * (1 - percent_second_label) + second_label * percent_second_label
#     )

#     # Combine the noise and the labels and run inference with the generator.
#     noise_and_labels = tf.concat([interpolation_noise, interpolation_labels], 1)
#     fake = trained_gen.predict(noise_and_labels)
#     return fake

def generate_image(trained_gen,number_generation:int):
    latent_dim = 128
    interpolation_noise = tf.random.normal(shape=(1, latent_dim))
    interpolation_noise = tf.repeat(interpolation_noise, repeats=number_generation)
    interpolation_noise = tf.reshape(interpolation_noise, (number_generation, latent_dim))


    labels = tf.zeros((number_generation,1))
    noise_and_labels = tf.concat([interpolation_noise, (labels)], 1)
    fake = trained_gen.predict(noise_and_labels)    
    return fake

trained_gen = cond_gan.generator
fake_images = generate_image(trained_gen,10)

# Here, we first sample noise from a normal distribution and then we repeat that for num_interpolation times 
# and reshape the result accordingly. We then distribute it uniformly for num_interpolation with the label 
# identities being present in some proportion.

fake_images *= 255.0
converted_images = fake_images.astype(np.uint8)
os.makedirs('generated_images', exist_ok=True)

for i in range(num_images):
    # Use imageio to save the image
    imageio.imsave(f'generated_images/fake_nonleak_image_{i}.png', converted_images[0])


InvalidArgumentError: {{function_node __wrapped__ConcatV2_N_2_device_/job:localhost/replica:0/task:0/device:GPU:0}} ConcatOp : Ranks of all input tensors should match: shape[0] = [10,128] vs. shape[1] = [10] [Op:ConcatV2] name: concat

In [5]:
latent_dim = 128
number_generation=10
interpolation_noise = tf.random.normal(shape=(1, latent_dim))
interpolation_noise = tf.repeat(interpolation_noise, repeats=number_generation)
interpolation_noise = tf.reshape(interpolation_noise, (number_generation, latent_dim))


labels = tf.zeros((number_generation,1))
noise_and_labels = tf.concat([interpolation_noise, labels], 1)